In [4]:
import pandas as pd
import numpy as np
from obspy import read
from obspy.signal.filter import bandpass
import os

# Load the CSV file
csv_file = './data/lunar/training/catalogs/initial_filtered_events_of_S12_GradeA_Catalog.csv'
df = pd.read_csv(csv_file)

# Create empty lists to store the data and labels
data_list = []
labels = []
aux_data_list = []  # To store auxiliary data (std dev before and after)

# Define the fixed length for the data (e.g., 14 minutes of data at a specific sampling rate)
fixed_length = 5565  # Example: 14 minutes of data at 100 Hz sampling rate

# Loop through each row in the CSV file
for idx, row in df.iterrows():
    filename = row['filename']
    time_rel = row['time_rel(sec)']  # Relative time point in seconds
    filter_low = row['Filter_low_freq_point']  # Low cut-off frequency
    filter_high = row['Filter_high_freq_point']  # High cut-off frequency
    label = row['label']  # Event label (0 or 1)

    # Load the MiniSEED file
    data_directory = './data/lunar/training/data/S12_GradeA/'
    mseed_file = f'{data_directory}{filename}.mseed'
    st = read(mseed_file)

    # Apply bandpass filter (between filter_low and filter_high)
    filtered_st = st.filter('bandpass', freqmin=filter_low, freqmax=filter_high)

    # Define the start and end times for slicing the data (4 minutes before and 10 minutes after time_rel)
    start_time = time_rel - 4 * 60  # 4 minutes before
    end_time = time_rel + 10 * 60  # 10 minutes after

    # Cut the data from start_time to end_time
    sliced_st = filtered_st.slice(starttime=st[0].stats.starttime + start_time, 
                                  endtime=st[0].stats.starttime + end_time)

    # Normalize the sliced data (mean=0, std=1)
    sliced_data = sliced_st[0].data
    sliced_data_normalized = (sliced_data - np.mean(sliced_data)) / np.std(sliced_data)

    # Calculate standard deviation before and after time_rel
    before_data = filtered_st.slice(starttime=st[0].stats.starttime + start_time, 
                                     endtime=st[0].stats.starttime + time_rel)
    after_data = filtered_st.slice(starttime=st[0].stats.starttime + time_rel, 
                                    endtime=st[0].stats.starttime + end_time)
    
    std_before = np.std(before_data[0].data)
    std_after = np.std(after_data[0].data)

    # Pad or truncate the data to the fixed length
    if len(sliced_data_normalized) > fixed_length:
        sliced_data_normalized = sliced_data_normalized[:fixed_length]
    else:
        sliced_data_normalized = np.pad(sliced_data_normalized, (0, fixed_length - len(sliced_data_normalized)), 'constant')

    # Store the normalized data, labels, and auxiliary data
    data_list.append(sliced_data_normalized)
    labels.append(label)
    aux_data_list.append([std_before, std_after])  # Append standard deviations

# Convert lists to NumPy arrays
X_data = np.array(data_list)
y_labels = np.array(labels)
aux_data = np.array(aux_data_list)  # Auxiliary data containing std devs

# Now, X_data contains the processed seismogram data,
# y_labels contains the labels, and aux_data contains the std devs
print(f"Data shape: {X_data.shape}, Labels shape: {y_labels.shape}, Auxiliary data shape: {aux_data.shape}")


Data shape: (589, 5565), Labels shape: (589,), Auxiliary data shape: (589, 2)


In [6]:
import numpy as np

# Assume X_data and y_labels are your arrays
np.save('X_data.npy', X_data)
np.save('y_labels.npy', y_labels)
np.save('aux_data.npy', aux_data)